In [15]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from datetime import timedelta
from datetime import datetime, time
import math


In [16]:
# calculate number of vehicles needed

def cal_veh_count(GSE_events):
 
    GPU_counts_narrow = 0
    GPU_counts_wide = 0
    aircraft_tractor_counts_narrow = 0
    aircraft_tractor_counts_wide = 0
    baggage_tractor_counts = 0
    cargo_loader_counts_wide = 0
    belt_loader_counts_narrow = 0
    belt_loader_counts_wide = 0
    cater_truck_counts = 0
    lav_truck_counts_narrow = 0
    lav_truck_counts_wide = 0
    water_truck_counts_narrow = 0
    water_truck_counts_wide = 0

    # GSE vehicles are shared within each airline
    airline_groups = [group for _,group in GSE_events.groupby('airline')]
    # wide-body and narrow-body aircraft use the same type of baggage tractors and catering trucks
    for i in range(len(airline_groups)):
        GSE_events_i = airline_groups[i]
        baggage_tractor_counts += GSE_events_i[GSE_events_i.GSE_type == 'baggage tractor'].veh_id.max()
        cater_truck_counts += GSE_events_i[GSE_events_i.GSE_type == 'catering truck'].veh_id.max()
    
    # wide-body and narrow-body aircraft use different types of other GSE vehicles
    narrow_body_events = GSE_events[GSE_events['aircraft_type'] == 'narrow']
    wide_body_events = GSE_events[GSE_events['aircraft_type'] == 'wide']
    narrow_body_events_airline_groups = [group for _,group in narrow_body_events.groupby('airline')]
    wide_body_events_airline_groups = [group for _,group in wide_body_events.groupby('airline')]

    for j in range(len(narrow_body_events_airline_groups)):
        GSE_events_narrow_j = narrow_body_events_airline_groups[j]
        GPU_counts_narrow += GSE_events_narrow_j[GSE_events_narrow_j.GSE_type == 'GPU'].veh_id.max()
        aircraft_tractor_counts_narrow += GSE_events_narrow_j[GSE_events_narrow_j.GSE_type == 'aircraft tractor'].veh_id.max()
        belt_loader_counts_narrow += GSE_events_narrow_j[GSE_events_narrow_j.GSE_type == 'belt loader'].veh_id.max()
        lav_truck_counts_narrow += GSE_events_narrow_j[GSE_events_narrow_j.GSE_type == 'lavatory truck'].veh_id.max()
        water_truck_counts_narrow += GSE_events_narrow_j[GSE_events_narrow_j.GSE_type == 'water truck'].veh_id.max()
    for k in range(len(wide_body_events_airline_groups)):
        GSE_events_wide_k = wide_body_events_airline_groups[k]
        GPU_counts_wide += GSE_events_wide_k[GSE_events_wide_k.GSE_type == 'GPU'].veh_id.max()
        aircraft_tractor_counts_wide += GSE_events_wide_k[GSE_events_wide_k.GSE_type == 'aircraft tractor'].veh_id.max()
        belt_loader_counts_wide += GSE_events_wide_k[GSE_events_wide_k.GSE_type == 'belt loader'].veh_id.max()
        lav_truck_counts_wide += GSE_events_wide_k[GSE_events_wide_k.GSE_type == 'lavatory truck'].veh_id.max()
        water_truck_counts_wide += GSE_events_wide_k[GSE_events_wide_k.GSE_type == 'water truck'].veh_id.max()
        cargo_loader_counts_wide += GSE_events_wide_k[GSE_events_wide_k.GSE_type == 'cargo loader'].veh_id.max()
        
    GSE_vehicle_count_list = {'baggage_tractor_counts': int(baggage_tractor_counts), 'cater_truck_counts': int(cater_truck_counts), \
                              'GPU_counts_narrow': int(GPU_counts_narrow), 'aircraft_tractor_counts_narrow': int(aircraft_tractor_counts_narrow), \
                              'belt_loader_counts_narrow': int(belt_loader_counts_narrow), 'lav_truck_counts_narrow': int(lav_truck_counts_narrow), \
                              'water_truck_counts_narrow': int(water_truck_counts_narrow), \
                              'GPU_counts_wide': int(GPU_counts_wide), 'aircraft_tractor_counts_wide': int(aircraft_tractor_counts_wide), \
                              'belt_loader_counts_wide': int(belt_loader_counts_wide), 'lav_truck_counts_wide': int(lav_truck_counts_wide), \
                              'water_truck_counts_wide': int(water_truck_counts_wide), 'cargo_loader_counts_wide': int(cargo_loader_counts_wide)
                               }
    return GSE_vehicle_count_list


In [17]:
test = pd.read_csv('./all_flight_GSE_events/S1_SOC_insufficient_40/ABE_GSE_events.csv')
veh_count = cal_veh_count(test)

In [18]:
veh_count

{'baggage_tractor_counts': 35,
 'cater_truck_counts': 21,
 'GPU_counts_narrow': 20,
 'aircraft_tractor_counts_narrow': 16,
 'belt_loader_counts_narrow': 17,
 'lav_truck_counts_narrow': 14,
 'water_truck_counts_narrow': 15,
 'GPU_counts_wide': 1,
 'aircraft_tractor_counts_wide': 1,
 'belt_loader_counts_wide': 1,
 'lav_truck_counts_wide': 1,
 'water_truck_counts_wide': 1,
 'cargo_loader_counts_wide': 2}

In [21]:
## Define a function to run each airport in parallel
def single_airport_veh_count(file_path):
    all_events = pd.read_csv(file_path, parse_dates=['start_time', 'end_time'])
    airport = os.path.basename(file_path).split('_')[0]
    # add aircraft_type information
    import glob
    GSE_tasks = pd.read_csv(glob.glob(f"./all_flight_GSE_tasks/*{airport}*.csv")[0])
    GSE_tasks = GSE_tasks[['airline', 'aircraft_type']].drop_duplicates()
    if 'aircraft_type' not in all_events.columns:
        all_events = pd.merge(all_events, GSE_tasks, on = 'airline', how = 'left')

    GSE_vehicle_count_list = cal_veh_count(all_events)
    GSE_vehicle_count_list['airport'] = airport
    return GSE_vehicle_count_list

In [22]:
# Parallelizing using Pool.map()
import multiprocessing as mp
import time

ts = time.time()
scenario_folders = ['S1_SOC_insufficient_40','S2_SOC_insufficient_20',
                   'S3_charge_during_service_gaps_40','S4_charge_during_service_gaps_20',
                   'S5_charge_overnight_40','S6_charge_overnight_20']

for scenario_folder in scenario_folders:
    folder_path = Path('./all_flight_GSE_events/{}'.format(scenario_folder))
    all_files = folder_path.glob('*.csv')

    pool = mp.Pool(mp.cpu_count())
     
    results = pool.map(single_airport_veh_count, list(all_files))
     
    pool.close()
    GSE_vehicle_count_df = pd.DataFrame(results)
    sce_short = scenario_folder.split('_')[0]
    GSE_vehicle_count_df.to_csv('./all_flight_GSE_vehicle_count/GSE_vehicle_count_{}.csv'.format(sce_short))
te = time.time()
print('Time Used: ', te-ts)

Time Used:  83.76994132995605
